##### Requirements

* ~~Step 1 -> Extract the invoice header & line items for all invoices.~~
  * A -> Add custom fields (like Loan#) that may be not part of standard fields that are extracted from Invoice Model
  * B -> For duplicates, identify that as "Duplicate" and include all extracted metadata/attributes from that service request/invoice.
  ~~* C -> Store the output in "JSON" (same format as excel spreadsheet as far as columns)~~ & SQL
* Step 1a -> Use Layout Model and create the output in "Markdown" and persist that data and apply RAG pattern.
* Step 1b -> Build custom model (potentially 2) to extract the data from Service Request/Invoice

* ~~Step 2 -> Extract the metadata (at line level) from the "Ground truth" -> Excel/CSV file (Standard with "Advances" worksheet)~~

* Step 3 -> Reconciliation Process
  * A -> Reconcile Invoice line items (from Step 1) against Step 2 with various techniques (Fuzzy Description match across all duplicated invoice, ~~matching key metadata~~ - invoice date, payment date, ~~service date~~, ~~invoice#~~, amount, property address, description and vendor name) and based on the outcome, ~~generate the exception report for all line items not matching/missing from extracted data~~.  ~~Create the matching report as different file.~~
  * B -> Reconcile Invoice line items (from Step 2) against Step 1 with various techniques (Fuzzy Description match across all duplicated invoice, ~~matching key metadata~~ - invoice date, payment date, ~~service date~~, ~~invoice#~~, amount, description and vendor name) and based on the outcome, ~~append the mismatch to existing exception report for all line items not matching from extracted data~~. (with indication of Step2 mismatch)

* Step 3a -> LLM Reconciliation Process
  * ~~Build a LLM Prompt such that it can compare JSON/SQL data identifying the match and exception based on key metadata called out earlier.~~
               
* Stretch Goal -> Build semantic model from the data that is stored in relation database & run PBI CoPilot on the top of that


In [1]:
import logging
import azure.functions as func
import os
import requests
import urllib.parse
from datetime import datetime, timedelta
from azure.storage.blob import generate_container_sas
from azure.identity import ManagedIdentityCredential, AzureCliCredential, ChainedTokenCredential
import json
import base64
import tiktoken

In [2]:
# Import Python libraries
import os
import openai
from openai import OpenAI, AzureOpenAI, AsyncAzureOpenAI
import pandas as pd  
import json


In [3]:
client = AzureOpenAI(
                    api_key = os.getenv('OpenAiWestUsKey'),  
                    api_version = os.getenv('OpenAiVersion'),
                    azure_endpoint = os.getenv('OpenAiWestUsEp')
                    )

#### Step 1 - Convert our Ground truth Excel file into JSON Output

In [4]:
def convertToJsonSerializable(data):  
    if pd.isnull(data):  
        # Convert NaN or NaT to None  
        return None  
    elif isinstance(data, pd.Timestamp):  
        # Convert Timestamp to ISO 8601 format string  
        return data.date().isoformat()
    else:  
        # Return other data types unchanged  
        return data 

#### Pre-req from previous process - Need the Loan# (as folder) and excel File as same name as Loan# in the folder
#### Within the folder, all sub-folders with Invoice/PDF will be processed

In [5]:
loanNumber = "4000835968"
dataDirectory = "../Data/Loan/" + loanNumber + "/"
CorpAdv = "CORPADV/"
Invoice = "INVOICE/"

In [6]:
excelFile = dataDirectory + loanNumber + ".xlsx"
outputJson = dataDirectory + "/Python/" + loanNumber + ".json"

# Load the Excel file  
xlsx = pd.ExcelFile(excelFile)  
  
# Create a dictionary to store the data from each sheet  
data = {}  
  
# Iterate through each worksheet in the Excel file  
for sheetName in xlsx.sheet_names:
    if sheetName != "Advances" and sheetName != "Summary":
        continue
    
    df = pd.read_excel(xlsx, sheetName)

    # Apply the conversion function to each cell in the DataFrame  
    df = df.applymap(convertToJsonSerializable)

    # Convert the DataFrame to a dictionary
    if sheetName == "Advances":
        df = df.rename(columns={'Payment Date': 'Payment_Date', 'Service Date': 'Service_Date', 'Invoice Number': 'Invoice_Nbr', 
                        'Expense Description': 'Item_Description', 'Amount Paid': 'Item_Amount'})
        subsetDf = df[['Payment_Date', 'Service_Date', 'Invoice_Nbr', 'Item_Description', 'Item_Amount']]
        filteredDf = subsetDf[subsetDf["Item_Amount"] > 0]
        filteredDf = filteredDf.reset_index()
        filteredDf = filteredDf.rename(columns={"index":"Row_Id"})
        data = filteredDf.to_dict(orient='records')
    #else:
    #    data[sheetName] = df.to_dict(orient='records')

# Convert the dictionary to a JSON string 
json_data = json.dumps(data, indent=4, ensure_ascii=False)
  
# Optionally, you can save this JSON data to a file  
with open(outputJson, 'w') as json_file:  
    json_file.write(json_data)  
  
print(f'JSON file has been created: {outputJson}')

JSON file has been created: ../Data/Loan/4000835968//Python/4000835968.json


In [7]:
# import csv  
# import json  
  
# invoiceSample = "./Data/Invoice/0084518695.csv"
  
# # Replace 'output.json' with the desired path for the JSON output file  
# outputJson = './Data/Invoice/0084518695.json'  
  
# # Read the CSV and convert it to a dictionary  
# data = []  
# with open(invoiceSample, mode='r', encoding='utf-8') as csvFile:  
#     reader = csv.DictReader(csvFile)  
#     for row in reader:  
#         if any(field.strip() for field in row.values()):  # Check for non-blank rows  
#             data.append(row)
  
# # Write the dictionary to a JSON file  
# with open(outputJson, mode='w', encoding='utf-8') as jsonFile:  
#     json.dump(data, jsonFile, indent=4)  
  
# print(f'JSON file has been created: {outputJson}')  


#### Step 2 - Invoke the Document Intelligence - Invoice Pre-built Model including the Key-Value Pairs

In [8]:
import json
import time
from requests import get, post
from azure.core.credentials import AzureKeyCredential
from azure.ai.formrecognizer import DocumentAnalysisClient
import re
import ast


In [9]:
def replaceJsonPlaceHolders(json, values):
  # find all placeholders
  placeholders = re.findall('<[\w ]+>', json)
  assert len(placeholders) == len(values), "Please enter the values of all placeholders."

  # replaces all placeholders with values
  for k, v in values.items():
      placeholder = "<%s>" % k
      json = json.replace(placeholder, v)

  return json

In [10]:
def replaceJsonPlaceHolder(json, values):
  # replaces all placeholders with values
  for k, v in values.items():
      placeholder = "<%s>" % k
      json = json.replace(placeholder, str(v))

  return json

In [11]:
class DateEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, datetime.date):
            return obj.isoformat()
        return super().default(obj)

In [12]:
sampleDocs = []
# sampleDocs = [
#     './Data/Invoice/4000835968/CORPADV/CORPADV_451102775.pdf',
#     './Data/Invoice/4000835968/CORPADV/CORPADV_451102776.pdf',
#     './Data/Invoice/4000835968/CORPADV/CORPADV_458680339.pdf',
#     './Data/Invoice/4000835968/CORPADV/CORPADV_458680340.pdf',
#     './Data/Invoice/4000835968/CORPADV/CORPADV_462460604.pdf',
#     './Data/Invoice/4000835968/CORPADV/CORPADV_462460605.pdf',
#     './Data/Invoice/4000835968/CORPADV/CORPADV_467091874.pdf',
#     './Data/Invoice/4000835968/CORPADV/CORPADV_467091875.pdf',
#     './Data/Invoice/4000835968/CORPADV/CORPADV_467091876.pdf',
# ]
# sampleDocs = [
#     '../Data/Loan/4000835968/Python/CORPADV_451102775.pdf',
# ]
sampleOutputDocs = []
if not os.path.exists(dataDirectory + CorpAdv):
    sourcePath = dataDirectory
else:
    sourcePath = dataDirectory + "CORPADV/"

if not os.path.exists(dataDirectory + Invoice):
    sourcePathInv = dataDirectory
else:
    sourcePathInv = dataDirectory + "INVOICE/"
    
destinationPath = dataDirectory + "Python/"
for file in os.listdir(sourcePath):
    if file.endswith(".pdf"):
        sampleDocs.append(sourcePath + file)

for file in os.listdir(sourcePathInv):
    if file.endswith(".pdf"):
        sampleDocs.append(sourcePathInv + file)

for file in os.listdir(destinationPath):
    if file.endswith(".json"):
        sampleOutputDocs.append(destinationPath + file)

In [13]:
docIntelligenceEndPoint = os.getenv('FormRecognizerEndPoint')
docIntelligenceKey = os.getenv('FormRecognizerKey')

In [14]:
def analyzeInvoice(pathAndFile):
    postUrl = docIntelligenceEndPoint + "documentintelligence/documentModels/prebuilt-invoice:analyze?api-version=2023-10-31-preview"
    postUrl = postUrl + "&stringIndexType=utf16CodeUnit&pages=1&queryFields=Loan&features=keyValuePairs%2CqueryFields"

    #print(postUrl)

    headers = {
        'Content-Type': 'application/octet-stream',
        'Ocp-Apim-Subscription-Key': docIntelligenceKey
    }

    params = {
        "includeTextDetails": True,
        "pages" : 1,
        "features":["keyValuePairs","queryFields"]

    }

    with open(pathAndFile, "rb") as f:
        dataBytes = f.read()

    try:
        response = post(url=postUrl, data=dataBytes, headers=headers)
        if response.status_code != 202:
            print("POST Analyze failed")
            return None
        #print("POST analyze succedded", response.headers["Operation-Location"])
        getUrl = response.headers['Operation-Location']
        #print(getUrl)
    except Exception as e:
        print("POST analyzed failed" + str(e))
        return None
    
    nTries = 50
    nTry = 0
    waitSec = 6

    while nTry < nTries:
        try:
            getResponse  = get(url=getUrl, headers=headers)
            respJson = json.loads(getResponse.text)
            if (getResponse.status_code != 200):
                print("Invoice Get Failed")
                return None
            status = respJson["status"]
            #print(status)
            if status == "succeeded":
                fileName = os.path.basename(pathAndFile).replace(".pdf", ".json")
                with open(destinationPath + fileName, "w") as f:
                    json.dump(respJson, f, indent=4, default=str)
                return respJson
            if status == "failed":
                print("Analysis Failed")
                return None
            time.sleep(waitSec)
            nTry += 1
        except Exception as e:
            print("Exception during GET" + str(e))
            return None

    

In [15]:
from decimal import Decimal
def processAnalyzeResult(analyzeResults, rowId):
    paymentDate = ''
    serviceDate = ''
    invoiceNbr = ''
    vendorName = ''
    expenseDesc = ''
    amountPaid = ''
    notes = ''
    loanNumber = ''
    invoiceTotal = ''
    invoiceDate = ''
    filledItems = []
    #templateStructure = '{"Payment Date":"<Payment_Date>","Service Date":"<Service_Date>", "Expense Type": "","Additional Expense Comments":"<Additional_Expense_Comments>","Expense Description": "<Expense_Description>","Amount Paid": "<Amount_Paid>","Amount Claimed": "","Amount Not Claimed": "","Unclaimed Amount Reason": "","Vendor Name": "<Vendor_Name>","Invoice Number": "<Invoice_Number>","Fee Type Code": "","Recovery Type": "","Actual Recovery Code": "","Expense Code": "","Fee Reference Comments": "","File Name": "","Document Available": "","Notes": "<Notes>"}'
    templateStructure = '{"Row_Id":"<Row_Id>", "Payment_Date":"<Payment_Date>","Service_Date":"<Service_Date>", "Loan_Number": "<Loan_Number>", "Item_Description": "<Item_Description>","Item_Amount": "<Item_Amount>","Invoice_Nbr": "<Invoice_Nbr>", "Invoice_Date": "<Invoice_Date>", "Invoice_Total": "<Invoice_Total>"}'
    for idx, invoice in enumerate(analyzeResults["documents"]):
        for name, field in invoice["fields"].items():
            if name != "Items":
                if name == "VendorName":
                    vendorName = field["content"]
                if name == "InvoiceId":
                    invoiceNbr = field["content"]
                if name == "InvoiceDate" and field["type"] == "date":
                    try:
                        invoiceDate = field["valueDate"]
                    except:
                        invoiceDate = field["content"]
                if name == "InvoiceTotal" and field["type"] == "currency":
                    try:
                        invoiceTotal = field["valueCurrency"]["amount"]
                    except:
                        invoiceTotal = field["content"]
                if name == "CustomerAddress":
                    notes = field["content"]
                if name == "Loan":
                    try:
                        loanNumber = field["content"]
                    except:
                        loanNumber = ''

                #print("...{}: {} has confidence {}".format(name, field.content, field.confidence))

        for idx, item in enumerate(invoice["fields"].get("Items").get("valueArray")):
            #print("...Item #{}".format(idx))
            for name, field in item["valueObject"].items():
                if name == "Amount" and field["type"] == "currency":
                    try:
                        amountPaid = field["valueCurrency"]["amount"]
                    except:
                        amountPaid = field["content"]
                if name == "Date" and field["type"] == "date":
                    try:
                        serviceDate = field["valueDate"]
                    except:
                        serviceDate = field["content"]
                    #print(serviceDate)
                if name == "Description":
                    expenseDesc = field["content"]

            values = {'Row_Id':rowId, 'Payment_Date':paymentDate,'Service_Date': serviceDate, 'Loan_Number': loanNumber, 
                'Item_Description': expenseDesc, 'Item_Amount': Decimal(str(amountPaid)), 'Invoice_Nbr': invoiceNbr.removeprefix("60"),
                'Invoice_Date': invoiceDate, 'Invoice_Total': invoiceTotal}
            filledItem = replaceJsonPlaceHolder(templateStructure,values)
            filledItems.append(filledItem)
            rowId += 1

                #print("......{}: {} has confidence {}".format(name, field.content, field.confidence))

    # for idx, kv in enumerate(analyzeResults["keyValuePairs"]):
    #     if (kv["key"] != None):
    #         #if (kv["key"]["content"] and kv["value"]["content"]):
    #         if (kv["key"]["content"]):
    #             if kv["key"]["content"] == "PaymentDate":
    #                 paymentDate = kv["value"]["content"]
    #             #print("Key...{}: Value...{}".format(kv.key.content, kv.value.content))
    

    #print(values)
    return filledItems

In [16]:
extractedData = []

docAnalysisClient = DocumentAnalysisClient(
        endpoint=docIntelligenceEndPoint, credential=AzureKeyCredential(docIntelligenceKey))

rowId = 0
#output = {"Advances": []}
for sampleDoc in sampleDocs:
    fileName = os.path.basename(sampleDoc).replace(".pdf", ".json")
    # Check if we already have ran the analysis
    #if os.path.exists(sampleDoc.replace(".pdf", ".json")):
    if os.path.exists(destinationPath + fileName):
        print("--------Process Already analyzed Invoice: ", sampleDoc)
        #with open(sampleDoc.replace(".pdf", ".json"), "r") as f:
        with open(destinationPath + fileName, "r") as f:
            invoices = json.load(f)
        analyzeResults = invoices['analyzeResult']
        filledItems = processAnalyzeResult(analyzeResults, rowId)
        for filledItem in filledItems:
            rowId += 1
            extractedData.append(ast.literal_eval(json.dumps(filledItem)))
        continue
    else:
        print("--------Process Invoice: ", sampleDoc)
        # with open(sampleDoc, "rb") as f:
        #     poller = docAnalysisClient.begin_analyze_document(
        #         "prebuilt-invoice", document=f, locale="en-US",
        #         pages="1", features=["keyValuePairs"]
        #     )
        # invoices = poller.result()
        # with open(destinationPath + fileName, "w") as f:
        #     json.dump(invoices.to_dict(), f, indent=4, default=str)
        analyzeInvoice(sampleDoc)
        with open(destinationPath + fileName, "r") as f:
            invoices = json.load(f)
        analyzeResults = invoices['analyzeResult']
        filledItems = processAnalyzeResult(analyzeResults, rowId)
        for filledItem in filledItems:
            rowId += 1
            extractedData.append(ast.literal_eval(json.dumps(filledItem)))


--------Process Already analyzed Invoice:  ../Data/Loan/4000835968/CORPADV/CORPADV_451102775.pdf
--------Process Already analyzed Invoice:  ../Data/Loan/4000835968/CORPADV/CORPADV_451102776.pdf
--------Process Already analyzed Invoice:  ../Data/Loan/4000835968/CORPADV/CORPADV_458680339.pdf
--------Process Already analyzed Invoice:  ../Data/Loan/4000835968/CORPADV/CORPADV_458680340.pdf
--------Process Already analyzed Invoice:  ../Data/Loan/4000835968/CORPADV/CORPADV_462460604.pdf
--------Process Already analyzed Invoice:  ../Data/Loan/4000835968/CORPADV/CORPADV_462460605.pdf
--------Process Already analyzed Invoice:  ../Data/Loan/4000835968/CORPADV/CORPADV_467091874.pdf
--------Process Already analyzed Invoice:  ../Data/Loan/4000835968/CORPADV/CORPADV_467091875.pdf
--------Process Already analyzed Invoice:  ../Data/Loan/4000835968/CORPADV/CORPADV_467091876.pdf
--------Process Already analyzed Invoice:  ../Data/Loan/4000835968/CORPADV/CORPADV_474710375.pdf
--------Process Already analyz

In [36]:
updatedOutput = json.dumps(extractedData, indent=4, ensure_ascii=False)
updatedOutput = updatedOutput.replace("\"{", "{")
updatedOutput = updatedOutput.replace("}\"", "}")
updatedOutput = updatedOutput.replace("\\", "")
tmpDf = pd.read_json(updatedOutput)
tmpDf['Invoice_Total'] = pd.to_numeric(tmpDf['Invoice_Total'])
tmpDf = tmpDf.sort_values(['Invoice_Nbr', 'Invoice_Date', "Item_Amount", 'Service_Date'], ascending=False)
dropDuplicate = tmpDf.drop_duplicates(['Item_Amount','Invoice_Date','Invoice_Nbr'], keep='first')
processedOutputJson = dataDirectory + "Python/" + loanNumber + "_FrOut.json"
processedOutputFullJson = dataDirectory + "Python/" + loanNumber + "_FullFrOut.json"
json.dumps(dropDuplicate.to_json(processedOutputJson, orient='records'), indent=2)
json.dumps(tmpDf.to_json(processedOutputFullJson, orient='records'), indent=2)
# Optionally, you can save this JSON data to a file  
#with open(processedOutputJson, 'w') as json_file: 
#    json_file.write(updatedOutput)

'null'

#### STEP 3A - Compare the Ground Truth JSON with the Document Intelligence JSON Output - Using "Python" Code
##### Ensure that the output JSON is in the same format as the Ground Truth JSON for both Matching and Non-Matching Rows

In [37]:
# Function to load and parse a JSON file  
def loadJson(file_path):
    with open(file_path, 'r') as file:  
        return json.load(file)

In [38]:
# Function to compare two lists of dictionaries based on key fields  
def compareJsonArray(jsonArray1, jsonArray2, keyFields):  
    #matchingOutput = {"Advances": []}
    #nonMatchingOutput = {"Advances": []}
    matchingObjects = []  
    nonMatchingObjects = []  
  
    # Convert the second JSON array to a dictionary for faster lookup  
    json_dict2 = {tuple(item[key] for key in keyFields): item for item in jsonArray2}
    json_dict1 = {tuple(item[key] for key in keyFields): item for item in jsonArray1}
  
    # Iterate through the first JSON array and compare  
    for item1 in jsonArray1:  
        key = tuple(item1[key] for key in keyFields)  
        item2 = json_dict2.get(key)  
        if item2:  
            # If a matching object is found based on key fields, store it  
            #matchingObjects.append({'object1': item1, 'object2': item2})
            item1.update({'Row_Id_Invoice': item2['Row_Id'], 'Invoice_Date': item2['Invoice_Date'], 'Invoice_Total': item2['Invoice_Total'],
                          'Invoice_Item_Description': item2['Item_Description']})
            matchingObjects.append(item1)
        else:  
            # If no matching object is found, store the non-matching object from the first array  
            nonMatchingObjects.append(item1)  
  
    # Also check for any objects in the second array that didn't match any in the first  
    for item2 in jsonArray2:  
        key = tuple(item2[key] for key in keyFields)  
        if key not in json_dict1:  
            nonMatchingObjects.append(item2)  
  
    #matchingOutput["Advances"] = matchingObjects
    #nonMatchingOutput["Advances"] = nonMatchingObjects
    #return matchingOutput, nonMatchingOutput  
    return matchingObjects, nonMatchingObjects

In [39]:
jsonArray1 = loadJson(dataDirectory + "Python/" + loanNumber + ".json")
jsonArray2 = loadJson(dataDirectory + "Python/" + loanNumber + "_FrOut.json")

#### Calculate the total amount of the invoices line item & the total amount of the ground truth and compare the two

In [40]:
groundTruthAmount = sum(map(lambda x: int(x['Item_Amount']), jsonArray1))
scannedInvoiceAmount = sum(map(lambda x: int(x['Item_Amount']), jsonArray2))
groundTruthAmount, scannedInvoiceAmount

(19421, 20996)

In [41]:
# matching, nonMatching = compareJsonArray(jsonArray1['Advances'], jsonArray2['Advances'], 
#                  ['Invoice_Nbr', "Service_Date"])
matching, nonMatching = compareJsonArray(jsonArray1, jsonArray2, 
                 ['Invoice_Nbr', "Service_Date", "Item_Amount"])
# Output the results  
#print("Matching Objects:")  
#print(json.dumps(matching, indent=4))  
#print("\nNon-Matching Objects:")  
#print(json.dumps(nonMatching, indent=4))  

In [42]:
### Incase if required to find duplicates

# import hashlib

# matchingFiltered = []
# md5List = []

# for item in matching["Advances"]:
#     md5Result = hashlib.md5(json.dumps(item, separators=(',', ':')).encode("utf-8")).hexdigest()
#     if md5Result not in md5List:
#         md5List.append(md5Result)
#         matchingFiltered.append(item)

In [43]:
matchingOutputJson = dataDirectory + "Python/" + loanNumber + "_FuzzyMatching.json"
nonMatchingOutputJson = dataDirectory + "Python/" + loanNumber + "_FuzzyNonMatching.json"
# Optionally, you can save this JSON data to a file  
with open(matchingOutputJson, 'w') as json_file:
    json_file.write(json.dumps(matching, indent=4))

with open(nonMatchingOutputJson, 'w') as json_file:
    json_file.write(json.dumps(nonMatching, indent=4))

#### Create the Data from from Excel Output and Document Intelligence API

In [38]:
excelDf = pd.read_json(dataDirectory + "Python/" + loanNumber + ".json")
frOutDf = pd.read_json(dataDirectory + "Python/" + loanNumber + "_FrOut.json")

#### STEP 3b - Invoke LLM and build Custom Prompt that can be used to Compare the Ground Truth JSON with the Document Intelligence JSON Output

In [155]:
def truncateToken(string: str, encoding_name: str, max_length: int = 128000) -> str:
    """Truncates a text string based on max number of tokens."""
    encoding = tiktoken.encoding_for_model(encoding_name)
    encoded_string = encoding.encode(string)
    num_tokens = len(encoded_string)

    if num_tokens > max_length:
        string = encoding.decode(encoded_string[:max_length])

    return string

def getMessagesFromHistory(systemPrompt: str, userConv: str):
        #messageBuilder = MessageBuilder(systemPrompt, modelId)
        messages = []
        messages.append({'role': 'system', 'content': systemPrompt})
        userContent = truncateToken(string=userConv, encoding_name="cl100k_base", max_length=128000)
        messages.append({'role': "user", 'content': userContent})

In [156]:
# systemTemplate = """Compare the provided JSON data arrays and determine the matching and non-matching rows based on key fields. Display the JSON output for both the matching and non-matching objects.
#                     Context:
#                     You have been given two JSON data arrays, Array A and Array B. Your task is to compare these arrays and identify the rows that match and do not match based on key fields. 
#                     The key fields in the JSON data arrays are "Invoice Number", "Service Date".
                    
#                     Instructions:
#                     Compare Array A and Array B based on specified key fields.
#                     Identify and display the matching rows in JSON format.
#                     Identify and display the non-matching rows in JSON format.
#                     Ensure that the output clearly indicates whether a row is matching or non-matching.
                    
#                     Do not include any explanations, only provide a  RFC8259 compliant JSON response following this format without deviation.
#                     [{
#                         "Payment Date": "Payment_Date",
#                         "Service Date": "2022-02-10",
#                         "Expense Type": "Expense_Type",
#                         "Additional Expense Comments": "Comments here",
#                         "Expense Description": "Description here",
#                         "Amount Paid": "Amount here",
#                         "Amount Claimed": "Amount Claimed here",
#                         "Amount Not Claimed": "Amount Not Claimed here",
#                         "Unclaimed Amount Reason": "",
#                         "Vendor Name": "Vendor Name here",
#                         "Invoice Number": "Invoice Number here",
#                         "Fee Type Code": "",
#                         "Recovery Type": "",
#                         "Actual Recovery Code": "",
#                         "Expense Code": "",
#                         "Fee Reference Comments": "",
#                         "File Name": "",
#                         "Document Available": "",
#                         "Notes": "Notes here"
#                         }]
#                         """

#### Run the LLM to compare two JSON data arrays

In [157]:
systemTemplate = """Compare the provided JSON data arrays and determine the matching rows based on key fields. Display the JSON output for the matching objects.
                    Context:
                    You have been given two JSON data arrays, Array A and Array B. Your task is to compare these arrays and identify the rows that match and do not match based on key fields. 
                    The key fields in the JSON data arrays are "Invoice Number", "Service Date".
                    
                    Instructions:
                    Compare Array A and Array B based on specified key fields.
                    Identify and display the matching rows in JSON format.

                    Array A is in the following format.
                    [{
                            "Payment_Date": "Payment_Date",
                            "Service_Date": "2022-02-10",
                            "Invoice_Nbr": "Invoice Number here",
                            "Item_Description": "",
                            "Item_Amount": ""
                    }]
                    
                    Array B is in the following format.
                    [{
                            "Payment_Date": "Payment_Date",
                            "Service_Date": "2022-02-10",
                            "Loan_Number": "Invoice Number here",
                            "Item_Description": "",
                            "Item_Amount": "",
                            "Invoice_Nbr": ""
                    }]

                    Do not include any explanations, only provide a  RFC8259 compliant JSON response following this format without deviation.
                    [{
                            "Payment_Date": "Payment_Date",
                            "Service_Date": "2022-02-10",
                            "Loan_Number": "Invoice Number here",
                            "Item_Description": "",
                            "Item_Amount": "",
                            "Invoice_Nbr": ""
                    }]
                    """

#### Because of the limitation on 4096 Tokens on the completion, let's break down the steps into multiple chunks

In [161]:
chunks = (len(jsonArray1) - 1) // 5 + 1
for i in range(chunks):
     processedOutputJson = dataDirectory + "Python/" + loanNumber + "_LlmMatching" + str(i) + ".json"
     if (os.path.exists(processedOutputJson)):
          continue
     #chunkedJsonArray1 = {"Advances": []}
     batch = jsonArray1[i*5:(i+1)*5]
     chunkedJsonArray1 = batch
     #print(chunkedJsonArray1)
     content = """Array A:{arrayA}
            Array B:{arrayB}
            """
     content = content.format(arrayA=json.dumps(chunkedJsonArray1, indent=4), arrayB=json.dumps(jsonArray2, indent=4))

     messages = []
     messages.append({'role': 'system', 'content': systemTemplate})
     #userContent = truncateToken(string=content, encoding_name="gpt-4-1106-preview", max_length=75000)
     userContent = content
     messages.append({'role': "user", 'content': userContent})

     completion = client.chat.completions.create(
          model=os.getenv('OpenAiGpt4Turbo'), 
          messages=messages,
          temperature=0,
          top_p=0,
          max_tokens=4096,
          n=1)
     answer = completion.choices[0].message.content
     answer = re.sub('\n', '', answer)
     answer = re.sub('```json', '', answer)
     answer = re.sub('```', '', answer)

     llmAnswer = json.loads(answer)
     # Optionally, you can save this JSON data to a file  
     with open(processedOutputJson, 'w') as json_file:  
          json_file.write(json.dumps(llmAnswer, indent=4))

In [162]:
# Combine all our LlmMatching Outputs to a single file
#llmMatchingOutput = {"Advances": []}
llmMatchingOutput = []
for i in range(chunks):
     with open(dataDirectory + "Python/" + loanNumber + "_LlmMatching" + str(i) + ".json", "r") as f:
          llmMatchingOutput.extend(json.load(f))

processedOutputJson = dataDirectory + "Python/" + loanNumber + "_LlmMatching.json"
# Optionally, you can save this JSON data to a file  
with open(processedOutputJson, 'w') as json_file:  
    json_file.write(json.dumps(llmMatchingOutput, indent=4))

In [163]:
#llmMatchingArray = loadJson(dataDirectory + "Python/" + loanNumber + "LlmMatching.json")
#fuzzyMatchingArray = loadJson(dataDirectory + "Python/" + loanNumber + "_FuzzyMatching.json")
#fuzzyNonMatchingArray = loadJson(dataDirectory + "Python/" + loanNumber + "_FuzzyNonMatching.json")
llmMatchingDf = pd.read_json(dataDirectory + "Python/" + loanNumber + "_LlmMatching.json")
fuzzyMatchingDf = pd.read_json(dataDirectory + "Python/" + loanNumber + "_FuzzyMatching.json")
fuzzyNonMatchingDf = pd.read_json(dataDirectory + "Python/" + loanNumber + "_FuzzyNonMatching.json")

#### Run LLM to find non-matching rows between two JSON data arrays

In [ ]:
# Compare the provided Tables and determine the matching rows based on key fields. Display the output for the matching objects.
    
# Context:
# You have been given two table, Table 1 and Table 2. Your task is to compare these tables and identify the rows that do match based on key fields. 
# The key fields in the Tables "Invoice Number", "Service Date".

# Instructions:
# Compare Table 1 and Table 2 based on specified key fields.
# Identify and display the matching rows from Table 1 and Table 2 and display the output with RowId from both tables

# Table 1
# RowId   Payment Date	Service Date	Invoice Number	Item Description	Item Amount
# 1   11/26/2021		CENG0122R35	PROPERTY INSPECTION FEE	15.00
# 2   12/28/2021		CENG0122R35	PROPERTY INSPECTION FEE	15.00
# 3   2/16/2022	2/10/2022	109661_00490	PROPERTY INSPECTION FEE	20.00
# 4   3/23/2022	3/17/2022	111413_00184	PROPERTY INSPECTION FEE	20.00
# 5   4/18/2022	4/13/2022	112727_00307	PROPERTY INSPECTION FEE	20.00
# 6   5/17/2022	5/12/2022	114129_00367	PROPERTY INSPECTION FEE	20.00
# 7   5/17/2022	5/12/2022	114129_00367	PROP PRES - PHOTOS	30.00
# 8   7/6/2022	7/1/2022	116869_00457	PROPERTY INSPECTION FEE	20.00
# 9   8/9/2022	8/4/2022	118626_00253	PROPERTY INSPECTION FEE	20.00

# Table 2
# RowId   Invoice Number	Loan Number	Invoice Date	Property Address	Item Description	Item/Service Date	Item Price	Invoice Total
# 1   110784761	4000835968	11/20/2021	3554 Branden Rd	Inspection Photos		0	15
# 2   110784761	4000835968	11/20/2021	3554 Branden Rd	Inspections		15	15
# 3   110784761	4000835968	11/20/2021	3554 Branden Rd	Inspection Photos		0	15
# 4   110784761	4000835968	11/20/2021	3554 Branden Rd	Inspections		15	15
# 5   110784761-0325503954	4000835968	11/20/2021	3554 Branden Rd	Property Services - Insp - Drive by Inspection	11/19/2021	15	15
# 6   111466643	4000835968	12/22/2021	3554 Branden Rd	Inspection Photos		0	15
# 7   111466643	4000835968	12/22/2021	3554 Branden Rd	Inspections		15	15
# 8   111466643-0326171164	4000835968	12/22/2021	3554 Branden Rd	Property Services - Insp - Drive by Inspection	12/22/2021	15	15
# 9   60109661_00490	4000835968	2/11/2022	3554 Branden Rd	Occupancy Inspection	2/10/2022	20	20
# 10  60109661_00490	4000835968	2/11/2022	3554 Branden Rd	Verify Occupancy	2/10/2022	20	20
# 11  60111413_00184	4000835968	3/18/2022	3554 Branden Rd	Occupancy Inspection	3/17/2022	20	20
# 12  60111413_00184	4000835968	3/18/2022	3554 Branden Rd	Foreclosure	3/17/2022	20	20
# 13  60112727_00307	4000835968	4/14/2022	3554 Branden Rd	Occupancy Inspection	4/13/2022	20	20


In [34]:
systemTemplate = """Compare the provided JSON data arrays and determine the non-matching rows based on key fields. Display the JSON output for the non-matching objects.
                    Context:
                    You have been given two JSON data arrays, Array A and Array B. Your task is to compare these arrays and identify the rows that do not match and do not match based on key fields. 
                    The key fields in the JSON data arrays are "Invoice Number", "Service Date".
                    
                    Instructions:
                    Compare Array A and Array B based on specified key fields.
                    Identify and display the non-matching rows in JSON format.
                    
                    Do not include any explanations, only provide a  RFC8259 compliant JSON response following this format without deviation.
                    {
                        "Advances":[{
                            "Payment Date": "Payment_Date",
                            "Service Date": "2022-02-10",
                            "Expense Type": "Expense_Type",
                            "Additional Expense Comments": "Comments here",
                            "Expense Description": "Description here",
                            "Amount Paid": "Amount here",
                            "Amount Claimed": "Amount Claimed here",
                            "Amount Not Claimed": "Amount Not Claimed here",
                            "Unclaimed Amount Reason": "",
                            "Vendor Name": "Vendor Name here",
                            "Invoice Number": "Invoice Number here",
                            "Fee Type Code": "",
                            "Recovery Type": "",
                            "Actual Recovery Code": "",
                            "Expense Code": "",
                            "Fee Reference Comments": "",
                            "File Name": "",
                            "Document Available": "",
                            "Notes": "Notes here"
                        }]
                    }
                    """

#### Statistics

In [35]:
excelData = dataDirectory + "Python/" + loanNumber + ".json"
FrData = dataDirectory + "Python/" + loanNumber + "_FrOut.json"
FrMatching = dataDirectory + "Python/" + loanNumber + "_FrMatching.json"
FrNoMatching = dataDirectory + "Python/" + loanNumber + "_FrNonMatching.json"
LlmMatching = dataDirectory + "Python/" + loanNumber + "_LlmMatching.json"
LlmNoMatching = dataDirectory + "Python/" + loanNumber + "_LlmNonMatching.json"

with open(excelData, 'r') as file:
    excelData = json.load(file)
    print("Total Excel Data: ", len(excelData['Advances']))

with open(FrData, 'r') as file:
    FrData = json.load(file)
    print("Total FrData: ", len(FrData['Advances']))

with open(FrMatching, 'r') as file:
    FrMatching = json.load(file)
    print("Total FrMatching: ", len(FrMatching['Advances']))

with open(FrNoMatching, 'r') as file:
    FrNoMatching = json.load(file)
    print("Total FrNoMatching: ", len(FrNoMatching['Advances']))

with open(LlmMatching, 'r') as file:
    LlmMatching = json.load(file)
    print("Total LlmMatching: ", len(LlmMatching['Advances']))

# with open(LlmNoMatching, 'r') as file:
#     LlmNoMatching = json.load(file)
#     print("Total LlmNoMatching: ", len(LlmNoMatching['Advances']))


Total Excel Data:  63
Total FrData:  52
Total FrMatching:  23
Total FrNoMatching:  40
Total LlmMatching:  29
